# Исследование данных игрового мобильного приложения

Вы работаете в компании, которая разрабатывает мобильные игры. К вам пришел менеджер с рядом задач по исследованию нескольких аспектов мобильного приложения.

**Задачи:**
1. В первую очередь, его интересует показатель retention. Напишите функцию для его подсчета.
2. Помимо этого, в компании провели A/B тестирование наборов акционных предложений. На основе имеющихся данных определите, какой набор можно считать лучшим и на основе каких метрик стоит принять правильное решение.  
3. Предложите метрики для оценки результатов последнего прошедшего тематического события в игре.  

**Предоставленные данные:**  
* `problem1-reg_data.csv` – данные о времени регистрации
* `problem1-auth_data.csv` – данные о времени захода пользователей в игру  
* `problem2.csv` – результаты A/B теста

## Этапы выполнения проекта

1. [Показатель Retention](#1.-Показатель-Retention)  
   - [1.1 Задача](#1.1-Задача)
   - [1.2 Загрузка данных](#1.2-Загрузка-данных)
   - [1.3 Предобработка данных](#1.3-Предобработка-данных)
   - [1.4 Расчет Retention](#1.4-Расчет-Retention)
2. [A/B тестирование](#2.-A/B-тестирование)
   - [2.1 Задача](#2.1-Задача)
3. [Набор метрик для оценки результатов](#3.-Набор-метрик-для-оценки-результатов)

In [1]:
# импорт библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

## 1. Показатель Retention

### 1.1 Задача

**Retention** – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). 

### 1.2 Загрузка данных

Изучим данные, предоставленные для проекта.

#### Данные о времени регистрации

Прочитаем файл `problem1-reg_data.csv` и сохраним его в переменной reg.

In [2]:
# чтение файла с данными с сохранением в reg
reg = pd.read_csv('problem1-reg_data.csv', sep=';')

Получение первых 10 строк таблицы.

In [3]:
# получение первых 10 строк таблицы.
reg.head(10)

,reg_ts,uid
0,911382223,1
1,932683089,2
2,947802447,3
3,959523541,4
4,969103313,5
5,977206495,6
6,984222671,7
7,990407778,8
8,995943765,9
9,1000951674,10


Общая информация о данных таблицы reg.

In [4]:
# получение общей информации о данных в таблице df
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   reg_ts  1000000 non-null  int64
 1   uid     1000000 non-null  int64
dtypes: int64(2)
memory usage: 15.3 MB


Рассмотрим полученную информацию подробнее.

Всего в таблице 2 столбца и 1000000 строк. Присутствует один тип данных: int64.

Подробно разберём, какие столбцы в reg содержат какую информацию:
* reg_ts — время регистриации в формате Unix timestamp
* uid — unique identifier (id пользователя)

Количество значений в столбцах одинаковое, пропущенные значения отсутствуют.

#### Данные о времени захода пользователей в игру

Прочитаем файл `problem1-auth_data.csv` и сохраним его в переменной reg.

In [5]:
# чтение файла с данными с сохранением в auth
auth = pd.read_csv('problem1-auth_data.csv', sep=';')

Получение первых 10 строк таблицы.

In [6]:
# получение первых 10 строк таблицы.
auth.head(10)

,auth_ts,uid
0,911382223,1
1,932683089,2
2,932921206,2
3,933393015,2
4,933875379,2
5,934372615,2
6,934662633,2
7,935002586,2
8,935141232,2
9,935682752,2


Общая информация о данных таблицы reg.

In [7]:
# получение общей информации о данных в таблице df
auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype
---  ------   -----
 0   auth_ts  int64
 1   uid      int64
dtypes: int64(2)
memory usage: 146.5 MB


In [8]:
# проверка наличия пропущенных значений.
auth.isna().sum()

auth_ts    0
uid        0
dtype: int64

In [9]:
# получим количество строк в таблице
auth.shape[0]

9601013

Рассмотрим полученную информацию подробнее.

Всего в таблице 2 столбца и 9601013 строк. Присутствует один тип данных: int64.

Подробно разберём, какие столбцы в auth содержат какую информацию:
* auth_ts — время захода пользователя в игру в формате Unix timestamp
* uid — unique identifier (id пользователя)

Количество значений в столбцах одинаковое, пропущенные значения отсутствуют.

#### Выводы

Каждая строка таблицы содержит id пользователя и временную отметку о его действии (регистрации или времени захода в игру). Пропуски в данных отсутствуют.

На следующем этапе необходимо будет:
* преобразовать временные отметки из формата Unix timestamp,
* сконвертировать uid в строковый формат (исключить возможные потери нулей и дальнейшие возможные проблемы, правда за счет некоторой незначительной для данного объема данных потери эффективности)  
* исследовать данные на наличие аномалий (например, регистрации с повторным присвоением уже использованного uid, аномальную частоту заходов пользователей в игру (отсечь ботов) и т.д.)

### 1.3 Предобработка данных

Сделаем преобразование временных отметок, сконвертируем uid в строковый формат и исследуем данные на наличие аномалий.

#### Данные о времени регистрации

**Сделаем преобразование столбца с временными отметками из Unix timestamp.**

In [10]:
# выполним преобразование
reg['reg_ts'] = pd.to_datetime(reg['reg_ts'], unit='s')

In [11]:
# посмотрим результаты преобразования
reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   reg_ts  1000000 non-null  datetime64[ns]
 1   uid     1000000 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 15.3 MB


Ознакомимся с распределением диапазона дат.

In [12]:
# посмотрим описательную статистику
reg.reg_ts.describe(include='all', datetime_is_numeric=True)

count                          1000000
mean     2019-01-24 12:12:34.523629056
min                1998-11-18 09:43:43
25%         2018-06-03 13:55:11.500000
50%                2019-07-30 02:35:25
75%      2020-04-01 16:28:40.499999744
max                2020-09-23 15:17:24
Name: reg_ts, dtype: object

Видим, что полный диапазон дат имеет значения от 1998-11-18 до 2020-09-23. Нижняя граница диапазона выглядит сомнительно, т.к. у мы разрабатываем мобильные игры.

Проверим процентое соотношение количества регистраций в тот или иной год.

In [13]:
# сгруппируем данные по года и посчитаем количество регистраций в процентах
reg_by_year = reg.reg_ts.dt.year.value_counts(normalize=True) \
                 .reset_index() \
                 .rename(columns={'index': 'year', 'reg_ts': 'regs_pct'}) 

In [14]:
# посмотрим на года с количеством регистраций больше одного процента
reg_by_year[reg_by_year['regs_pct'] >= 0.01]

,year,regs_pct
0,2020,0.354963
1,2019,0.291102
2,2018,0.159729
3,2017,0.087645
4,2016,0.048187
5,2015,0.026344
6,2014,0.014455


Дальше будем работать с данными начиная с 2014 года. Остальные данные будем считать выбросами и исключим из таблицы.

In [15]:
# сохраним данные в переменную reg_14
reg_14 = reg[reg.loc[:, 'reg_ts'] > '2014-01-01 00:00:00.000000000']

In [16]:
# сбросим индексы
reg_14.reset_index(inplace=True, drop=True)

In [17]:
# посмотрим описательную статистику
reg_14.reg_ts.describe(include='all', datetime_is_numeric=True)

count                           982425
mean     2019-03-09 11:11:10.245470208
min                2014-01-01 00:03:11
25%                2018-07-04 19:51:14
50%                2019-08-09 16:52:29
75%                2020-04-05 05:43:58
max                2020-09-23 15:17:24
Name: reg_ts, dtype: object

In [18]:
# посмотрим количество отсеченных строк
reg.shape[0] - reg_14.shape[0]

17575

In [19]:
# количество отсеченных строк в процентном соотношении
round((reg.shape[0] - reg_14.shape[0]) * 100 / reg.shape[0], 2)

1.76

Видим, что данные раньше 2014 года были отсечены (17575 строк или 1.76%).

**Преобразуем значения uid в строковый формат данных**

In [20]:
# преобразование типа данных столбца uid в строковые значения
reg_14.loc[:, ['uid']] = reg_14['uid'].astype('str')

/home/nktnlx/yes/lib/python3.7/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [21]:
# проверим верность выполнения
reg_14.dtypes

reg_ts    datetime64[ns]
uid               object
dtype: object

Преобразование uid в строковый формат было выполнено успешно.

**Проверим наличие регистраций пользователей с повторным присвоением уже использованного uid**

In [22]:
# отобразим количество встреченных дубликатов uid
reg_14.uid.duplicated().sum()

0

Повторного использования уже выданных uid не обнаружено.

#### Данные о времени захода пользователей в игру

**Сделаем преобразование столбца с временными отметками из Unix timestamp.**

In [23]:
# выполним преобразование
auth['auth_ts'] = pd.to_datetime(auth['auth_ts'], unit='s')

In [24]:
# посмотрим результаты преобразования
auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype         
---  ------   -----         
 0   auth_ts  datetime64[ns]
 1   uid      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 146.5 MB


Ознакомимся с распределением диапазона дат.

In [25]:
# посмотрим описательную статистику
auth.auth_ts.describe(include='all', datetime_is_numeric=True)

count                          9601013
mean     2019-01-29 16:12:06.254660096
min                1998-11-18 09:43:43
25%                2018-06-07 12:40:47
50%                2019-07-31 12:43:16
75%                2020-04-02 10:25:21
max                2020-09-23 15:17:24
Name: auth_ts, dtype: object

Видим, что полный диапазон дат имеет значения от 1998-11-18 до 2020-09-23.

Как и для случая с регистрациями оставим только значения начиная с 2014 года. Остальные данные будем считать выбросами и исключим из таблицы.

In [26]:
# сохраним данные в переменную auth_14
auth_14 = auth[auth.loc[:, 'auth_ts'] > '2014-01-01 00:00:00.000000000']

In [27]:
# сбросим индексы
auth_14.reset_index(inplace=True, drop=True)

In [28]:
# посмотрим описательную статистику
auth_14.auth_ts.describe(include='all', datetime_is_numeric=True)

count                          9445996
mean     2019-03-10 11:48:49.205278464
min                2014-01-01 00:03:11
25%         2018-07-05 22:59:18.500000
50%                2019-08-10 05:59:40
75%      2020-04-05 16:38:24.249999872
max                2020-09-23 15:17:24
Name: auth_ts, dtype: object

In [29]:
# посмотрим количество отсеченных строк
auth.shape[0] - auth_14.shape[0]

155017

In [30]:
# количество отсеченных строк в процентном соотношении
round((auth.shape[0] - auth_14.shape[0]) * 100 / auth.shape[0], 2)

1.61

Видим, что данные раньше 2014 года были отсечены (155017 строк или 1.61%).

**Преобразуем значения uid в строковый формат данных**

In [31]:
# преобразование типа данных столбца uid в строковые значения
auth_14.loc[:, ['uid']] = auth_14['uid'].astype('str')

In [32]:
# проверим верность выполнения
auth_14.dtypes

auth_ts    datetime64[ns]
uid                object
dtype: object

Преобразование uid в строковый формат было выполнено успешно.

**Изучим статистику заходов пользователей в игру.**

In [33]:
# группировка по пользователям и подсчет количества заходов в игру
auth_freq = auth_14.groupby('uid', as_index=False) \
                   .agg({'auth_ts': 'count'}) \
                   .rename(columns={'auth_ts': 'visits'}) \
                   .sort_values('visits', ascending=False)

In [34]:
# описательная статистика распределения числа заходов пользователей в игру
auth_freq.visits.describe()

count    983392.000000
mean          9.605525
std          45.489029
min           1.000000
25%           1.000000
50%           1.000000
75%           1.000000
max         647.000000
Name: visits, dtype: float64

Максимальное количество заходов в игру сильно отличается от среднего и медианного значения. Проверим время между заходами, чтобы исключить заходы ботов.

In [35]:
# группировка по пользователям и сортировка по времени захода
auth_14_delta = auth_14.reset_index() \
                       .groupby(['index', 'uid', 'auth_ts'], as_index=False) \
                       .agg({'index': 'count'}) \
                       .drop(columns='index') \
                       .sort_values(['uid', 'auth_ts'])

In [36]:
# подсчет разницы во времени между заходами
auth_14_delta['delta'] = auth_14_delta.auth_ts.diff(periods=1)

auth_14_delta

,uid,auth_ts,delta
690733,100000,2016-09-20 02:33:46,NaT
8485653,1000000,2020-07-21 19:58:14,1400 days 17:24:28
8485662,1000001,2020-07-21 19:59:12,0 days 00:00:58
8553022,1000001,2020-07-26 12:14:31,4 days 16:15:19
8593456,1000001,2020-07-29 07:45:01,2 days 19:30:30
...,...,...,...
9097164,999999,2020-09-01 04:42:45,3 days 12:16:06
9151803,999999,2020-09-04 18:14:23,3 days 13:31:38
9231540,999999,2020-09-09 22:14:16,5 days 03:59:53
9328627,999999,2020-09-16 03:59:54,6 days 05:45:38


In [37]:
# убираем всех у кого между сессиями меньше 5 минут 
# (возможные боты и пользователи не ставшие играть / случайные заходы в игру)
auth_14_delta = auth_14_delta[auth_14_delta['delta'] >= '0 days 00:05:00']

In [38]:
# сформируем перечень "нормальных" пользователей (не боты и не случайные заходы)
active_uids = auth_14_delta.uid.unique()

In [39]:
# уберем uid с аномальным числом заходов в игру из auth_14
auth_14_cleaned = auth_14.query('uid in @active_uids')

In [40]:
# посмотрим количество отсеченных строк
auth_14.shape[0] - auth_14_cleaned.shape[0]

645083

In [41]:
# количество отсеченных строк в процентном соотношении
round((auth_14.shape[0] - auth_14_cleaned.shape[0]) * 100 / auth_14.shape[0], 2)

6.83

Было отброшено 645083 аномальных строк (по частоте заходов в игру), что составляет 6.83%.

#### Выводы

Была произведена смена форматов данных (Unix timestamp --> date time; uid из целочисленых в строковые значения), проверка на дублирующие значения uid после регистрации (дубликатов выявлено не было), удаление не релевантных по дате данных (оставили данные начиная с 2014 года), а так же поиск и удаление аномальных значений по частоте захода в игру (удалено порядка 7% данных -- возможно боты или случайные заходы пользователей в игру, без самого процесса игры).

На следующем этапе необходимо будет посчитать retention пользователей.

### 1.4 Расчет Retention

Посчитаем retention (уровень удержания) игроков по дням от даты регистрации игрока. 

## 2. A/B тестирование

### 2.1 Задача

## 3. Набор метрик для оценки результатов